# Import

In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt

from qick import *
from qick.helpers import gauss
from tqdm import tqdm_notebook as tqdm

import time
import os
import sys
sys.path.append('/home/xilinx/jupyter_notebooks/')
sys.path.append('C:\\_Lib\\python\\rfsoc\\rfsoc_multimode\\example_expts')
# sys.path.append('C:\\_Lib\\python\\multimode')
import scipy as sp
import json
from scipy.fft import fft, fftfreq

from slab.instruments import *
from slab.experiment import Experiment
from slab.datamanagement import SlabFile
from slab import get_next_filename, get_current_filename, AttrDict

# Figure params
plt.rcParams['figure.figsize'] = [10,6]
plt.rcParams.update({'font.size': 14})

Warning serial library import failed.
Could not load InstrumentManagerWindow
Could not import QickInstrument, try install qick package
Could not load AD5780 dac 
Could not load labbrick
Could not load Autonics TM4 Driver


In [2]:
import json
import os
# path = os.getcwd()
# path = r'H:\Shared drives\SLab\Multimode\experiment\250418' # for data 
path = r'H:\Shared drives\SLab\Multimode\experiment\250505_craqm' # for data 
print("path: ", path)

path:  H:\Shared drives\SLab\Multimode\experiment\250505_craqm


# setup experiment hardware

In [3]:
expt_path=path+'\\data'

print('Data will be stored in', expt_path)

Data will be stored in H:\Shared drives\SLab\Multimode\experiment\250505_craqm\data


In [4]:
curr_path = os.getcwd()
# config_file = curr_path+'\\configs\\config_rfsoc_ziqian.yml'
config_file = curr_path+'\\configs\\hardware_config_202505.yml'
config_path = config_file
print('Config will be', config_path)

exp_param_file = curr_path+'\\configs\\experiment_config.yml'
exp_param_path = exp_param_file
print('Exp Param will be', exp_param_path)

Config will be c:\_Lib\python\multimode_expts\configs\hardware_config_202505.yml
Exp Param will be c:\_Lib\python\multimode_expts\configs\experiment_config.yml


In [5]:
qubit_i = 0
import yaml
with open(config_file, 'r') as cfg_file:
    yaml_cfg = yaml.safe_load(cfg_file)
yaml_cfg = AttrDict(yaml_cfg)

In [6]:
im = InstrumentManager(ns_address='192.168.137.25') # SLAC lab

In [7]:
im.keys()

dict_keys(['Pyro.NameServer', 'Qick101'])

In [8]:
print(im['Qick101'])

<Pyro4.core.Proxy at 0x251fc972fd0; not connected; for PYRO:obj_65d5568241f84ce6b086ad472fc0670a@192.168.137.101:38179>


In [9]:
soc = QickConfig(im[yaml_cfg['aliases']['soc']].get_cfg())
print(soc)

QICK running on ZCU216, software version 0.2.291

Firmware configuration (built Tue Oct 15 12:06:51 2024):

	Global clocks (MHz): tProcessor 430.080, RF reference 245.760

	7 signal generator channels:
	0:	axis_signal_gen_v6 - envelope memory 32768 samples (5.128 us)
		fs=6389.760 MHz, fabric=399.360 MHz, 32-bit DDS, range=6389.760 MHz
		DAC tile 2, blk 0 is 0_230, on JHC3
	1:	axis_signal_gen_v6 - envelope memory 32768 samples (5.128 us)
		fs=6389.760 MHz, fabric=399.360 MHz, 32-bit DDS, range=6389.760 MHz
		DAC tile 2, blk 1 is 1_230, on JHC4
	2:	axis_signal_gen_v6 - envelope memory 32768 samples (5.128 us)
		fs=6389.760 MHz, fabric=399.360 MHz, 32-bit DDS, range=6389.760 MHz
		DAC tile 2, blk 2 is 2_230, on JHC3
	3:	axis_signal_gen_v6 - envelope memory 32768 samples (5.128 us)
		fs=6389.760 MHz, fabric=399.360 MHz, 32-bit DDS, range=6389.760 MHz
		DAC tile 2, blk 3 is 3_230, on JHC4
	4:	axis_signal_gen_v6 - envelope memory 32768 samples (4.762 us)
		fs=6881.280 MHz, fabric=430.080 MH

In [10]:
soc.cycles2us(1)

0.0023251488095238095

# choose experiment name

In [39]:
import sys

# Path to add
expts_path = 'C:\\_Lib\\python\\multimode_expts'

# Add the path to the system path at the highest priority
# if expts_path not in sys.path:
sys.path.insert(0, expts_path)
print('Path added at highest priority')

# Verify the path is added
print(sys.path)

# Import the experiments module from multimode
import experiments as meas

# Verify the module is imported from the correct path
print(meas.__file__)

Path added at highest priority
['C:\\_Lib\\python\\multimode_expts', 'C:\\_Lib\\python\\multimode_expts', 'C:\\_Lib\\python\\multimode_expts', 'C:\\_Lib\\python', 'C:\\_Lib\\python\\slab', 'C:\\_Lib\\python\\qick\\qick_lib', 'C:\\_Lib\\python\\rfsoc\\rfsoc_multimode\\example_expts', 'c:\\_Lib\\python\\multimode_expts', 'c:\\Users\\26049\\.conda\\envs\\slab\\python38.zip', 'c:\\Users\\26049\\.conda\\envs\\slab\\DLLs', 'c:\\Users\\26049\\.conda\\envs\\slab\\lib', 'c:\\Users\\26049\\.conda\\envs\\slab', '', 'C:\\Users\\26049\\AppData\\Roaming\\Python\\Python38\\site-packages', 'c:\\Users\\26049\\.conda\\envs\\slab\\lib\\site-packages', 'c:\\Users\\26049\\.conda\\envs\\slab\\lib\\site-packages\\easy_lmfit-1.0-py3.8.egg', 'c:\\Users\\26049\\.conda\\envs\\slab\\lib\\site-packages\\attrdict-2.0.1-py3.8.egg', 'c:\\Users\\26049\\.conda\\envs\\slab\\lib\\site-packages\\win32', 'c:\\Users\\26049\\.conda\\envs\\slab\\lib\\site-packages\\win32\\lib', 'c:\\Users\\26049\\.conda\\envs\\slab\\lib\\site

# Run experiments

In [45]:
# notuse expts
# experiment_class, experiment_name = 'single_qubit.single_shot_old', 'HistogramExperiment_oldold'
# experiment_class, experiment_name = 'single_qubit.single_shot', 'HistogramExperiment_old'
# experiment_class, experiment_name = 'qubit_cavity.non_demolution_readout', 'DemolutionMeasurementExperiment'


# experiment_class, experiment_name = 'single_qubit.phase_sweep', 'PhaseSweepExperiment'
# experiment_class, experiment_name = 'single_qubit.phase_sweep_averager', 'PhaseSweepAveragerExperiment'
# experiment_class, experiment_name = 'qubit_cavity.ecd', 'ECDExperiment'
# experiment_class, experiment_name = 'qubit_cavity.ecd_read_spec', 'ECDReadSpecExperiment'
# experiment_class, experiment_name = 'single_qubit.pulse_probe_ef_spectroscopy', 'PulseProbeEFSpectroscopyExperiment'
# experiment_class, experiment_name = 'qubit_cavity.single_mode_wigner_tomography', 'WignerTomography1ModeExperiment'
# experiment_class, experiment_name = 'qubit_cavity.single_mode_cats_wigner_tomography', 'WignerTomography1ModeCatExperiment'
# experiment_class, experiment_name = 'single_qubit.amplitude_rabi_general', 'AmplitudeRabiGeneralExperiment'
# experiment_class, experiment_name = 'single_qubit.amplitude_rabi_ef', 'AmplitudeRabiEFExperiment'
# experiment_class, experiment_name = 'single_qubit.t2_echo', 'RamseyEchoExperiment'
# experiment_class, experiment_name = 'single_qubit.t2_cavity', 'CavityRamseyExperiment_old'

# experiment_class, experiment_name = 'single_qubit.cavity2qubit', 'Cavity2QubitSweepExperiment'

In [46]:
# ---------------------- Readout ----------------------
# experiment_class, experiment_name = 'single_qubit.resonator_spectroscopy', 'ResonatorSpectroscopyExperiment' # for readout

# ---------------------- Qubit ----------------------
## Basic
# experiment_class, experiment_name = 'single_qubit.t1', 'T1Experiment'
experiment_class, experiment_name = 'single_qubit.single_shot', 'HistogramExperiment'

# experiment_class, experiment_name = 'single_qubit.single_shot_prepulse','HistogramPrepulseExperiment'
# experiment_class, experiment_name = 'single_qubit.t2_ramsey', 'RamseyExperiment'
## Advanced
# experiment_class, experiment_name = 'single_qubit.amplitude_rabi', 'AmplitudeRabiExperiment' # <-------------- for ge and ef
# experiment_class, experiment_name = 'single_qubit.length_rabi_general', 'LengthRabiGeneralExperiment'
# experiment_class, experiment_name = 'single_qubit.pulse_probe_spectroscopy', 'PulseProbeSpectroscopyExperiment'
# experiment_class, experiment_name = 'single_qubit.pulse_probe_ef_spectroscopy', 'PulseProbeEFSpectroscopyExperiment'
# experiment_class, experiment_name = 'single_qubit.pulse_probe_spectroscopy_prepulse', 'PulseProbePrePulseSpectroscopyExperiment'

# ---------------------- Man Cavity ----------------------

# experiment_class, experiment_name = 'single_qubit.cavity_spectroscopy', 'CavitySpectroscopyExperiment'
# experiment_class, experiment_name = 'single_qubit.length_rabi_f0g1_general', 'LengthRabiGeneralF0g1Experiment'
# experiment_class, experiment_name = 'single_qubit.t1_cavity', 'T1CavityExperiment'
# experiment_class, experiment_name = 'single_qubit.t2_cavity', 'CavityRamseyExperiment' # -------
# experiment_class, experiment_name = 'single_qubit.pulse_probe_f0g1_spectroscopy', 'PulseProbeF0g1SpectroscopyExperiment' # <--------------

# ---------------------- Storage ----------------------
# experiment_class, experiment_name = 'single_qubit.rf_flux_spectroscopy_f0g1', 'FluxSpectroscopyF0g1Experiment' 
# experiment_class, experiment_name = 'single_qubit.sideband_general', 'SidebandGeneralExperiment'
# experiment_class, experiment_name = 'single_qubit.sideband_t1_general', 'SidebandT1GeneralExperiment' # --------

# ----------------------- Storage RB -----------------------
# experiment_class, experiment_name = 'single_qubit.rb_BSgate_postselection', 'SingleBeamSplitterRBPostSelection'

## Wigner Tomography 
# experiment_class, experiment_name = 'single_qubit.parity_delay', 'ParityDelayExperiment'
# experiment_class, experiment_name = 'single_qubit.parity_gain', 'ParityGainExperiment'
# experiment_class, experiment_name = 'single_qubit.parity_freq', 'ParityFreqExperiment'

## Cavity Temperature 
# experiment_class, experiment_name = 'single_qubit.parity_measurement_temp', 'ParityTempExperiment'



# experiment_class, experiment_name = 'single_qubit.rb_ziqian', 'SingleRB'
# experiment_class, experiment_name = 'single_qubit.rbam', 'MultiRBAM'
# experiment_class, experiment_name = 'single_qubit.rb_BSgate', 'SingleBeamSplitterRB'
# experiment_class, experiment_name = 'single_qubit.rb_BSgate_postselection', 'SingleBeamSplitterRBPostSelection'
# experiment_class, experiment_name = 'single_qubit.rb_BSgate_check_target', 'SingleBeamSplitterRB_check_target'
# experiment_class, experiment_name = 'single_qubit.rb_BSgate_ramsey', 'SingleBeamSplitterRB_ramsey'
# experiment_class, experiment_name = 'qubit_cavity.displacement_enhanced_sideband', 'DisplacementEnhancedSidebandExperiment'

# experiment_class, experiment_name = 'qubit_cavity.out_and_back', 'OutAndBackExperiment'
# experiment_class, experiment_name = 'qubit_cavity.amplitude_rabi_f0g1', 'AmplitudeRabiF0g1Experiment'

# experiment_class, experiment_name = 'single_qubit.dragpulse_calibration', 'DragCalibrationExperiment' 
# experiment_class, experiment_name = 'single_qubit.fast_reset_test', 'FastResetExperiment'
# experiment_class, experiment_name = 'single_qubit.active_reset_single_shot', 'HistogramActiveResetExperiment'


# 

run_exp = eval(f"meas.{experiment_class}.{experiment_name}(soccfg=soc, path=expt_path, prefix=experiment_name, config_file=config_path)")


with open(exp_param_file, 'r') as file:

    # Load the YAML contents
    loaded = yaml.safe_load(file)
    run_exp.cfg.expt = eval(f"loaded['{experiment_name}']")

# special updates on device_config file
run_exp.cfg.device.readout.relax_delay = [2500] # Wait time between experiments [us]
print(run_exp.cfg)
run_exp.go(analyze=False, display=False, progress=True, save=True)

{'device': {'qubit': {'T1': [370.2930190133963], 'f_ge': [3569.4263053206314], 'f_ef': [3426.287416084909], 'ramp_sigma': [0.005], 'ramp_sigma_num': [3.0], 'pulses': {'pi_ge': {'gain': [11165], 'sigma': [0.035], 'length': [0.0], 'type': 'gauss'}, 'hpi_ge': {'gain': [5289], 'length': [0.0], 'sigma': [0.035], 'type': 'gauss'}, 'pi_ef': {'gain': [8590], 'length': [0.0], 'sigma': [0.035], 'type': 'gauss'}, 'hpi_ef': {'gain': [4060], 'sigma': [0.035], 'type': 'gauss'}}, 'T1_ef': [167.5647128005103]}, 'readout': {'trig_offset': 450, 'frequency': [748.8], 'gain': [800], 'phase': [0.8473846167680859], 'threshold': [-148.02027429647023], 'threshold_list': [[-148.02027429647023]], 'man1_drive_list': [0.0], 'man2_drive_list': [0.0], 'readout_length': [1], 'relax_delay': [2500], 'confusion_matrix_with_reset': [0.9964530022802125, 0.0035469977197875036, 0.013768302080657598, 0.9862316979193424], 'confusion_matrix_without_reset': [0.9880000000000001, 0.0119999999999999, 0.025599999999999998, 0.9744]

  0%|          | 0/10000 [00:00<?, ?it/s]

  0%|          | 0/10000 [00:00<?, ?it/s]

Saving H:\Shared drives\SLab\Multimode\experiment\250505_craqm\data\00035_HistogramExperiment.h5


In [14]:
soc.us2cycles(0.03, gen_ch = 1)

12

In [21]:
for i in range(1): 
    print('Run %d' % i) 

Run 0


In [68]:
# quick pulse maker 
with open(config_file, 'r') as cfg_file:
    yaml_cfg = yaml.safe_load(cfg_file)
yaml_cfg = AttrDict(yaml_cfg)

from MM_dual_rail_base import MM_dual_rail_base
mm_base_dummy = MM_dual_rail_base(yaml_cfg)
# prep_stor = mm_base_dummy.prep_random_state_mode(2,1)  # prepare the storage state + 
# prep_man_hpi = mm_base_dummy.prep_random_state_mode(3, 1)[:-1]  # prepare the storage state +
prep_man_pi = mm_base_dummy.prep_man_photon(1)  # prepare the storage state +
mm_base_dummy.get_prepulse_creator(   prep_man_pi[:-1]).pulse.tolist()


[[3569.462965491148, 3426.2579958470365],
 [11161, 8483],
 [0.0, 0.0],
 [0, 0],
 [2, 2],
 ['g', 'g'],
 [0.035, 0.035]]

In [ ]:
prep_man

array([['qubit', 'ge', 'pi', '0'],
       ['qubit', 'ef', 'pi', '0'],
       ['man', 'M1', 'pi', '0']], dtype='<U11')

In [ ]:
# Sample NumPy array
my_array = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]]).tolist()

# Reverse the array along axis 0
reversed_array = np.flip(my_array, axis=0).tolist() + my_array

reversed_array

[[7, 8, 9], [4, 5, 6], [1, 2, 3], [1, 2, 3], [4, 5, 6], [7, 8, 9]]

$\text{Gain (linear)} = 10^{\frac{\text{dB}}{20}} $

# Run sequential experiment

In [20]:
import multimode_expts.sequential_experiment as seq_exp

experiment_name = 'length_rabi_sweep'
# experiment_name = 'dc_flux_sweep'
# experiment_name = 'length_rabi_f0g1_sweep'
# experiment_name = 'manipulate_spectroscopy_dc_flux_sweep'
# experiment_name = 'gain_displace_sweep'
# experiment_name = 'sideband_general_sweep' # Deprecated

# --------- Sideband Experiment -----------
# experiment_class, experiment_name = 'sidebands_class', 'sideband_general_sweep'
# experiment_class, experiment_name = 'sidebands_class', 'sideband_gain_freq_sweep'
# experiment_class, experiment_name = 'sidebands_class', 'sideband_cross_kerr_cancellation'
# experiment_name = 'single_shot_phase_sweep'
# experiment_name = 'single_shot_freq_sweep'
# experiment_name = 'HistogramExperiment_freq_sweep'
# experiment_name = 'HistogramExperiment_ef_freq_sweep'
# experiment_name = 'single_shot_time_sweep'
# experiment_name = 'single_qubit_tomography'      # <--------------
# experiment_name = 'SingleRB_sweep_freq'
# experiment_name = 'SingleRB_sweep_pi_amp'
# experiment_name = 'SingleRB_sweep_hpi_amp'
# experiment_name = 'SingleRB_sweep_f0g1_phase'
# experiment_name = 'SingleRB_sweep_de|pth_and_prepulse'
# experiment_name = 'SingleRB_sweep_depth'
# experiment_name = 'MultiRBAM_sweep_depth'
# experiment_name = 'SingleBeamsplitterRB_sweep_depth'
# experiment_name = 'SingleBeamSplitterRB_sweep_freq'
# experiment_name = 'SingleBeamSplitterRB_gain_freq'
# experiment_name = 'pulseprobe_f0g1_dc_flux_sweep'
# experiment_name = 'PhaseSweepAverager_sweep_reps_middlepulse'
# experiment_name = 'cavity_t1_dc_flux_sweep'
# experiment_name = 'displace_enhanced_sweep'
# experiment_name = 'fluxspectroscopy_f0g1_dc_flux_sweep'
# experiment_name = 'storage_sideband_sweep'
# experiment_name = 'sideband_fidelity_optimization'
# experiment_name = 'storage_t1_optimization'
# experiment_name = 'SingleBeamSplitterRBPostSelection_sweep_depth'#
# experiment_name = 'SingleBeamSplitterRBPostSelection_sweep_depth_and_ram'
# experiment_name = 'SingleBeamSplitterRBPostSelection_sweep_depth_and_single_spec'
# experiment_name = 'DualRail_sweep_depth_and_single_spec_and_stor'
# experiment_name = 'SingleBeamSplitterRBPostSelection_sweep_depth_storsweep'
# experiment_name = 'SingleBeamSplitterRBPostSelection_sweep_depth_defined_storsweep'

# experiment_name = 'ramsey_sweep'
# experiment_name = 'cavity_ramsey_sweep'
# experiment_name = 'manipulate_dc_flux_sweep'
# experiment_name = 'rb_bs_optimization'
# experiment_name = 'rb_bs_dual_rail_optimization'
# experiment_name = 'cross_kerr_sweep'
# experiment_name = 'single_dual_rail_tomography'
# experiment_name = 'two_dual_rail_tomography'
# experiment_name = 'cavity_t1_dc_flux_sweep_new'
# experiment_name = 'cavity_temperature_sweep'
# experiment_name = 'cavity_temperature_sweep_parity'
# ---------------------- Dual Rail ----------------------
# experiment_class, experiment_name = 'MM_dual_rail_seq_exp', 'SingleBeamSplitterRB_check_target_sweep_depth'
# experiment_class, experiment_name = 'MM_dual_rail_seq_exp', 'SingleBeamSplitterRB_stor_ramsey_spec'
# experiment_class, experiment_name = 'MM_dual_rail_seq_exp', 'SingleBeamSplitterRB_stor_ramsey_spec_for_sp_pairs'


# ---------------------- Cavity RTamsey ----------------------
# experiment_class, experiment_name = 'sweep_cavity_ramsey_expts', 'cavity_temperature_sweep_parity_with_without_reset'
# experiment_class, experiment_name = 'sweep_cavity_ramsey_expts', 'cavity_ramsey_with_spectators'
# experiment_class, experiment_name = 'sweep_cavity_ramsey_expts', 'stor_cavity_ramsey_sweep'

# ---------------------- storage sweep-----------
# experiment_class, experiment_name = 'storage_sweep_class', 'storage_t1_sweep'


# # Option 1
# class_for_exp = eval('seq_exp.' + experiment_class)()
# eval('class_for_exp.' + experiment_name)(soccfg=soc, path=expt_path, prefix=experiment_name, config_file=config_path, exp_param_file=exp_param_file)

# Option 2
eval('seq_exp.' + experiment_name)(soccfg=soc, path=expt_path, prefix=experiment_name, config_file=config_path, exp_param_file=exp_param_file)

#Option 3
# class_for_exp = eval('seq_exp.' + experiment_class)(soccfg=soc, path=expt_path, prefix=experiment_name, config_file=config_path, exp_param_file=exp_param_file)
# eval('class_for_exp.run_sweep')( sweep_experiment_name = experiment_name)


Config will be C:\_Lib\python\multimode_expts\configs\hardware_config_202505.yml
Index: 0 Freq. = 3420.0 GHz


TypeError: list indices must be integers or slices, not float

In [ ]:
experiment_class = 'sidebands_class'
class_for_exp = eval('seq_exp.' + experiment_class)(soccfg=soc, path=expt_path, prefix=experiment_name, config_file=config_path, exp_param_file=exp_param_file)

In [ ]:
class_for_exp.run_sweep

<bound method sidebands_class.run_sweep of <multimode_expts.sequential_experiment.sidebands_class object at 0x0000028220427700>>

In [ ]:
np.array([
                    349.66, # 349.415, 
                    521.23, #520.925,
                    696.98, #696.665,
                    876.38, #875.845,
                    1053.95, #1053.395,
                    1245.52, # 1245.5, #1245.16,
                    1423.27 #1423.04
                    ]) + 1

array([ 350.66,  522.23,  697.98,  877.38, 1054.95, 1246.52, 1424.27])

In [ ]:
class_for_exp.cavity_ramsey_with_spectators(soccfg=soc, path=expt_path, prefix=experiment_name, config_file=config_path, exp_param_file=exp_param_file)

TypeError: cavity_ramsey_with_spectators() got multiple values for argument 'soccfg'

In [ ]:
soc

In [ ]:
from multimode_expts.MM_rb_base import  *

f0g1_pulse_str = [['qubit', 'ge', 'pi', 0],
                    ['qubit', 'ef', 'pi', 0],
                    ['man', 'M1', 'pi', 0]]

mm_base = MM_rb_base(cfg = yaml_cfg)
creator = mm_base.get_prepulse_creator(f0g1_pulse_str)
creator.pulse.tolist()


[[3568.497301707094, 3425.2997709216247, 2007.54],
 [10973, 9530, 15000],
 [0, 0, 0.6164057205175526],
 [0, 0, 0],
 [2, 2, 0],
 ['g', 'g', 'flat_top'],
 [0.035, 0.035, 0.005]]

In [ ]:
from MM_base import MM_base
stor_no=1
man_idx=1
mm_base = MM_base(cfg = yaml_cfg)
pre_sweep_pulse_str = [
    ['man', 'M' + str(man_idx) , 'pi',0],
                       ['qubit', 'ef', 'pi',0]]
creator = mm_base.get_prepulse_creator(pre_sweep_pulse_str)
a1 = creator.pulse.tolist()
a1


In [ ]:
from MM_base import MM_base
stor_no=1
man_idx=1
mm_base = MM_base(cfg = yaml_cfg)
pre_sweep_pulse_str = [['storage', 'M'+ str(man_idx) + '-S' + str(stor_no), 'pi',0],
    ['qubit', 'ge', 'hpi',0], # Starting parity meas
                ['qubit', 'ge', 'parity_M' + str(man_idx),0], 
                ['qubit', 'ge', 'hpi',0]]
creator = mm_base.get_prepulse_creator(pre_sweep_pulse_str)
a1 = creator.pulse.tolist()
a1


[[349.415, 3568.1515953227604, 3568.1515953227604, 3568.1515953227604],
 [3000, 5510, 0, 5510],
 [0.9771428571428572, 0, 1.6833333333333333, 0],
 [0, 0, 0, 0],
 [1, 2, 2, 2],
 ['flat_top', 'g', 'const', 'g'],
 [0.005, 0.035, 0.0, 0.035]]

In [ ]:
from MM_base import MM_base
stor_no=[1,2]
man_idx=1
mm_base = MM_base(cfg = yaml_cfg)
pre_sweep_pulse_str = [['qubit', 'ge', 'pi',0],
                    ['qubit', 'ef', 'pi',0],
                    ['man', 'M' + str(man_idx) , 'pi',0],
                    ['storage', 'M'+ str(man_idx) + '-S' + str(stor_no[0]), 'hpi',0], 
                    ['storage', 'M'+ str(man_idx) + '-S' + str(stor_no[1]), 'hpi',0],
                    ['storage', 'M'+ str(man_idx) + '-S' + str(stor_no[1]), 'hpi',0]]

creator = mm_base.get_prepulse_creator(pre_sweep_pulse_str)
a1 = creator.pulse.tolist()
a1


[[3568.1751638611518, 3425.5564066067, 2007.06, 349.415, 520.925, 520.925],
 [10716, 8236, 15000, 3000, 5000, 5000],
 [0, 0, 0.651287, 0.4885714285714285, 0.565451994946885, 0.565451994946885],
 [0, 0, 0, 0, 0, 0],
 [2, 2, 0, 1, 1, 1],
 ['g', 'g', 'flat_top', 'flat_top', 'flat_top', 'flat_top'],
 [0.035, 0.035, 0.005, 0.005, 0.005, 0.005]]

In [ ]:
from MM_base import MM_base
stor_no=[[1,2],[3,4]]
man_idx=1
mm_base = MM_base(cfg = yaml_cfg)
pre_sweep_pulse_str = [['qubit', 'ge', 'pi',0],
                    ['qubit', 'ef', 'pi',0],
                    ['man', 'M' + str(man_idx) , 'pi',0],
                    ['storage', 'M'+ str(man_idx) + '-S' + str(stor_no[0][0]), 'hpi',0], 
                    ['storage', 'M'+ str(man_idx) + '-S' + str(stor_no[0][1]), 'hpi',0],
                    ['storage', 'M'+ str(man_idx) + '-S' + str(stor_no[0][1]), 'hpi',0],
                    ['qubit', 'ge', 'pi',0],
                    ['qubit', 'ef', 'pi',0],
                    ['man', 'M' + str(man_idx) , 'pi',0],
                    ['storage', 'M'+ str(man_idx) + '-S' + str(stor_no[1][0]), 'hpi',0], 
                    ['storage', 'M'+ str(man_idx) + '-S' + str(stor_no[1][1]), 'hpi',0], 
                    ['storage', 'M'+ str(man_idx) + '-S' + str(stor_no[1][1]), 'hpi',0]]

creator = mm_base.get_prepulse_creator(pre_sweep_pulse_str)
a1 = creator.pulse.tolist()
a1

[[3568.1751638611518,
  3425.5564066067,
  2007.06,
  349.415,
  520.925,
  520.925,
  3568.1751638611518,
  3425.5564066067,
  2007.06,
  696.665,
  875.845,
  875.845],
 [10716, 8236, 15000, 3000, 5000, 5000, 10716, 8236, 15000, 9000, 9000, 9000],
 [0,
  0,
  0.651287,
  0.4885714285714285,
  0.565451994946885,
  0.565451994946885,
  0,
  0,
  0.651287,
  0.666955,
  0.6271437312608766,
  0.6271437312608766],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [2, 2, 0, 1, 1, 1, 2, 2, 0, 1, 1, 1],
 ['g',
  'g',
  'flat_top',
  'flat_top',
  'flat_top',
  'flat_top',
  'g',
  'g',
  'flat_top',
  'flat_top',
  'flat_top',
  'flat_top'],
 [0.035,
  0.035,
  0.005,
  0.005,
  0.005,
  0.005,
  0.035,
  0.035,
  0.005,
  0.005,
  0.005,
  0.005]]

In [ ]:
stor_no=[3,4]
man_idx=1
post_selection_pulse_str = [['storage', 'M'+ str(man_idx) + '-S' + str(stor_no[0]), 'hpi',0], 
                    ['storage', 'M'+ str(man_idx) + '-S' + str(stor_no[0]), 'hpi',0],
                ['qubit', 'ge', 'hpi',0], # Starting parity meas
                ['qubit', 'ge', 'parity_M' + str(man_idx),0], 
                ['qubit', 'ge', 'hpi',0]]# measure]
creator = mm_base.get_prepulse_creator(post_selection_pulse_str)
I_gate1 = creator.pulse.tolist()
post_selection_pulse_str = [
                ['storage', 'M'+ str(man_idx) + '-S' + str(stor_no[1]), 'hpi',0], 
                    ['storage', 'M'+ str(man_idx) + '-S' + str(stor_no[1]), 'hpi',0],
                ['qubit', 'ge', 'hpi',0], # Starting parity meas
                ['qubit', 'ge', 'parity_M' + str(man_idx),0], 
                ['qubit', 'ge', 'hpi',0]]# measure]
creator = mm_base.get_prepulse_creator(post_selection_pulse_str)
I_gate2 = creator.pulse.tolist()


I_gate = [I_gate1, I_gate2]

post_selection_pulse_str = [['storage', 'M'+ str(man_idx) + '-S' + str(stor_no[0]), 'hpi',0], 
                    ['storage', 'M'+ str(man_idx) + '-S' + str(stor_no[0]), 'hpi',0],
                ['storage', 'M'+ str(man_idx) + '-S' + str(stor_no[1]), 'hpi',0], 
                ['qubit', 'ge', 'hpi',0], # Starting parity meas
                ['qubit', 'ge', 'parity_M' + str(man_idx),0], 
                ['qubit', 'ge', 'hpi',0]]
creator = mm_base.get_prepulse_creator(post_selection_pulse_str)
X_gate1 = creator.pulse.tolist()
post_selection_pulse_str = [['storage', 'M'+ str(man_idx) + '-S' + str(stor_no[1]), 'hpi',0], 
                    ['storage', 'M'+ str(man_idx) + '-S' + str(stor_no[1]), 'hpi',0],
                ['qubit', 'ge', 'hpi',0], # Starting parity meas
                ['qubit', 'ge', 'parity_M' + str(man_idx),0], 
                ['qubit', 'ge', 'hpi',0]]
creator = mm_base.get_prepulse_creator(post_selection_pulse_str)
X_gate2 = creator.pulse.tolist()

X_gate = [X_gate1, X_gate2]

post_selection_pulse_str = [['storage', 'M'+ str(man_idx) + '-S' + str(stor_no[0]), 'hpi',0], 
                    ['storage', 'M'+ str(man_idx) + '-S' + str(stor_no[0]), 'hpi',0],
                ['storage', 'M'+ str(man_idx) + '-S' + str(stor_no[1]), 'hpi',90], 
                ['qubit', 'ge', 'hpi',0], # Starting parity meas
                ['qubit', 'ge', 'parity_M' + str(man_idx),0], 
                ['qubit', 'ge', 'hpi',0]]
creator = mm_base.get_prepulse_creator(post_selection_pulse_str)
Y_gate1 = creator.pulse.tolist()
post_selection_pulse_str = [['storage', 'M'+ str(man_idx) + '-S' + str(stor_no[1]), 'hpi',0], 
                    ['storage', 'M'+ str(man_idx) + '-S' + str(stor_no[1]), 'hpi',0],
                ['qubit', 'ge', 'hpi',0], # Starting parity meas
                ['qubit', 'ge', 'parity_M' + str(man_idx),0], 
                ['qubit', 'ge', 'hpi',0]]
creator = mm_base.get_prepulse_creator(post_selection_pulse_str)
Y_gate2 = creator.pulse.tolist()

Y_gate = [Y_gate1, Y_gate2]

# Prepulse and Postpulse generator

## Dataset

In [106]:
from dataset import * 
from dataset import storage_man_swap_dataset


In [107]:
dataset = storage_man_swap_dataset('man1_storage_swap_dataset.csv')
dataset.df

,stor_name,freq (MHz),precision (MHz),pi (mus),h_pi (mus),gain (DAC units),last_update
0,M1-S1,349.110,NaN,0.813627,0.360513,3000,2025-05-22 16:05:21.623488
1,M1-S2,520.610,NaN,0.165878,0.080903,5000,2025-05-22 15:50:43.604290
2,M1-S3,696.350,NaN,1.122884,0.512333,9000,2025-05-22 15:50:34.895075
3,M1-S4,875.285,NaN,1.068157,0.488362,9000,2025-05-22 15:54:56.582975
4,M1-S5,1053.040,NaN,1.744303,0.833387,7500,2025-05-22 15:55:49.320754
5,M1-S6,1244.860,NaN,1.388454,0.652345,13500,2025-05-22 15:56:51.940958
6,M1-S7,1422.660,NaN,1.993553,0.948300,10000,2025-05-22 15:57:38.027389
7,M1-S8,-1.000,-1.00,-1.000000,-1.000000,-1,51:31.3
8,M1-S9,-1.000,-1.00,-1.000000,-1.000000,-1,51:31.3
9,M1-S10,-1.000,-1.00,-1.000000,-1.000000,-1,51:31.3


## Adding dump and coupler swaps to the dataset

In [ ]:
# ##[2313.3, 5000, , 2, 2], 
# #[2093.9, , 15, 2, 2] 
ds = dataset
# M1-D1
ds.append_dataset('M1-D1', 2313.3, 0.01, 10, 0, 5000)
#M2 - D2
ds.append_dataset('M2-D2', 2093.9, 0.01, 15, 0, 8000)
# M1-C (coupler)
ds.append_dataset('M1-C', 944.25, 0.01, 0.316677658, 0.158422064, 1000)

C:\_Lib\python\multimode_expts\dataset.py:92: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df = self.df.append(new_row, ignore_index=True)
C:\_Lib\python\multimode_expts\dataset.py:92: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df = self.df.append(new_row, ignore_index=True)
C:\_Lib\python\multimode_expts\dataset.py:92: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df = self.df.append(new_row, ignore_index=True)


## New Creator

In [108]:
from multimode_expts.MM_base import *
mm_base = MM_base(cfg = yaml_cfg)

man_idx = 1
stor_id = [1,2]

## Parity Meas
parity_str = [['qubit', 'ge', 'hpi', 0 ], # Starting parity meas
                ['qubit', 'ge', 'parity_M' + str(man_idx), 0 ], 
                ['qubit', 'ge', 'hpi', 0 ], 
                ]

# parity_str_multiple

## Prepapre Man1 photon
prep_man_pi = [
                ['qubit', 'ge', 'pi', 0],
                ['qubit', 'ef', 'pi', 0],
                ['man', 'M1', 'pi', 0]
            ]
post_f0g1_str_for_ramsey = [['man', 'M1', 'pi', 0],
                ['qubit', 'ef', 'pi', 0], 
                ['qubit', 'ge', 'hpi', 0]]
storage = [['storage', 'M1-S1', 'pi', 0]   ]

man = [['man', 'M1', 'pi', 0]   ]

prep_stor = [
                ['qubit', 'ge', 'hpi', 0],
                ['qubit', 'ef', 'pi', 0],
                # ['man', 'M1', 'pi', 0],
                # ['storage', 'M1-S3', 'pi', 0]  ,
                   ]

new_pulse =[['qubit', 'ge', 'pi', 0],
                    ['qubit', 'ef', 'pi', 0],
                    ['man', 'M1', 'pi', 0],
                    
                    # ['man', 'M1', 'pi', 0],
                    ['qubit', 'ef', 'pi', 0],
                    ['qubit', 'ge', 'pi', 0]]

    
creator = mm_base.get_prepulse_creator( new_pulse)
creator.pulse.tolist()
# pulse_str

[[3569.4198176358545,
  3426.2975766655427,
  2008.975,
  3426.2975766655427,
  3569.4198176358545],
 [11165, 8627, 15000, 8627, 11165],
 [0.0, 0.0, 0.6303030824111784, 0.0, 0.0],
 [0, 0, 0, 0, 0],
 [2, 2, 0, 2, 2],
 ['g', 'g', 'flat_top', 'g', 'g'],
 [0.035, 0.035, 0.005, 0.035, 0.035]]

In [ ]:
prep_stor[-1]

['storage', 'M1-S2', 'pi', 0]

In [27]:
3571 - 3428

143

# Ramp Current

In [45]:
# coupler
dcflux = YokogawaGS200(address="192.168.137.148")
dcflux.set_output(True)
dcflux.set_mode('current')
dcflux.ramp_current(0.000, sweeprate=0.0001)
dcflux.ramp_current(0.00032, sweeprate=0.0001)

In [ ]:
# jpa
dcflux = YokogawaGS200(address="192.168.137.149")
dcflux.set_output(True)
dcflux.set_mode('current')
# dcflux.ramp_current(0.000, sweeprate=0.0001)
dcflux.ramp_current(3.800, sweeprate=0.0001)

In [46]:
# jpa Signal core
from slab.instruments import SignalCore
sc = SignalCore(name = "SignalCore_JPA", address="10001E48")
sc.open_device()
sc.set_output_state(True)
sc.set_power(-11.67)
# sc.set_output_state(True)
sc.close_device()

1879597607920

dcflux = YokogawaGS200(address="192.168.137.148")
    dcflux.set_output(True)
    dcflux.set_mode('current')
    dcflux.ramp_current(0.000, sweeprate=0.002)